In [2]:
# Reload modules automatically
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_scatter import scatter_add
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops


In [ ]:
data_params = dict(
    graph_model = 'CSBM',
    classes = 2,
    n = 1000,
    n_per_class_trn = 400,
    K = 2,
    sigma = 1,
    avg_within_class_degree = 1.58 * 2,
    avg_between_class_degree = 0.37 * 2,
    inductive_samples = 1000
)
graph_model = create_graph_model(data_params)
X_np, A_np, y_np = graph_model.sample(data_params["n"], 0)
X = torch.tensor(X_np, dtype=torch.float32, device="cuda:0")
A = torch.tensor(A_np, dtype=torch.float32, device="cuda:0")
y = torch.tensor(y_np, device="cuda:0")
adj = SparseTensor.from_dense(A)
row, col, _ = adj.coo()
print(row)
print(col)
print(torch.stack((row, col), dim=0))
A.nonzero().t()

In [9]:
dataset = Planetoid(root='data', name='Cora')
X = dataset.data.x
y = dataset.data.y
edge_index = dataset.data.edge_index
print(edge_index)
edge_weight = torch.ones((edge_index.size(1), ), device=edge_index.device)
row, col = edge_index
deg = scatter_add(edge_weight, col, dim=0, dim_size=X.size(0))
print(deg)
print(float("inf"))
print(y.shape)
print(y.view(-1).shape)
#y = F.one_hot(y).to(torch.float)
#print(y.size())
print(F.one_hot(y).shape)
y_ = F.one_hot(y)
print(y_)
y_[[1,2]]
torch.zeros((3,3))

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
tensor([3., 3., 5.,  ..., 1., 4., 4.])
inf
torch.Size([2708])
torch.Size([2708])
torch.Size([2708, 7])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [56]:
class GCN(MessagePassing):
    def __init__(self, in_channels, out_channels, aggr: str = "add", 
                 flow: str = "source_to_target"):
        super().__init__(aggr, flow)
        self.lin = Linear(in_channels, out_channels, bias=False)

    def forward(self, X, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=X.size(0))
        X = self.lin(X)
        return self.propagate(edge_index, X=X)

    def message(self, X_j):
        print(X_j.size())
        print(X_j[:10, :5])
        return X_j + 1

    def update(self, aggr_out, X_j):
        print(aggr_out.size())
        return aggr_out * 100

In [57]:
model = GCN(in_channels=dataset.data.x.size(1), out_channels=10)
model.forward(dataset.data.x, dataset.data.edge_index)

torch.Size([13264, 10])
tensor([[-0.0173,  0.0529,  0.0042,  0.0244,  0.1070],
        [-0.0173,  0.0529,  0.0042,  0.0244,  0.1070],
        [-0.0173,  0.0529,  0.0042,  0.0244,  0.1070],
        [-0.0072,  0.1087,  0.0614,  0.0283,  0.0275],
        [-0.0072,  0.1087,  0.0614,  0.0283,  0.0275],
        [-0.0072,  0.1087,  0.0614,  0.0283,  0.0275],
        [-0.0455,  0.0201, -0.0049,  0.0073,  0.0557],
        [-0.0455,  0.0201, -0.0049,  0.0073,  0.0557],
        [-0.0455,  0.0201, -0.0049,  0.0073,  0.0557],
        [-0.0455,  0.0201, -0.0049,  0.0073,  0.0557]],
       grad_fn=<SliceBackward0>)
torch.Size([2708, 10])


tensor([[389.7239, 412.7906, 429.0812,  ..., 420.3212, 393.5004, 410.1242],
        [389.2765, 397.4312, 409.7231,  ..., 387.7980, 379.2740, 417.0523],
        [600.7773, 623.2421, 602.2294,  ..., 606.1293, 593.8380, 614.5193],
        ...,
        [202.6021, 203.6831, 204.2789,  ..., 216.3896, 203.2019, 191.3758],
        [531.6699, 533.6844, 491.4141,  ..., 524.5166, 515.8243, 479.3176],
        [516.1280, 524.0164, 496.5078,  ..., 513.3365, 534.8948, 487.5916]],
       grad_fn=<MulBackward0>)